引用
==========

在Clojure中使用标识与值分离的思想解决并发问题. 引用相当于一个指针, 指针可以指向不同的值, 而每个值本身不会变化.

使用ref创建引用, 使用deref解除引用(或者使用@宏)

In [3]:
(def user (ref {:name "alice", :age 12}))

#'user/user

In [4]:
(println user)
(println @user)

#ref[{:status :ready, :val {:name alice, :age 12}} 0x2c424287]
{:name alice, :age 12}


nil

ref-set
----------------

Clojure提供了多种方式修改引用的值, 这些方法都需要在`dosync`函数内执行.

`ref-set`直接修改引用的指向

In [12]:
(def user (ref {:name "alice", :age 12}))
(println "Before: " @user)
(dosync (ref-set user {}))
(println "After : " @user)

Before:  {:name alice, :age 12}
After :  {}


nil

alter
-------------

`alter`将读取引用的值, 修改值, 写入修改值三个操作合并到一起.

`alter`接受一个引用和一个函数, 将函数应用到应用的值上, 并将操作后的结果重新写入引用.


In [14]:
(def all-users (ref {}))
(defn new-user [id login budget]
    {:id id, :login login, :budget budget, :expenses 0})

(defn add-new-user [login budget]
    (dosync
        (let [current (count @all-users)
              user (new-user (inc current) login budget)]
            (alter all-users assoc login user))))

#'user/add-new-user

In [15]:
(add-new-user "alice", 120)

{"alice" {:id 1, :login "alice", :budget 120, :expenses 0}}

In [16]:
(add-new-user "bob", 240)

{"alice" {:id 1, :login "alice", :budget 120, :expenses 0}, "bob" {:id 2, :login "bob", :budget 240, :expenses 0}}

commute
----------

`commute`与`alter`的输入是一样的, 但与`alter`不同的地方在于: 

当多个线程同时修改引用时, `alter`会检查是否发生冲突, 并最终导致只有1个线程修改成功, 其余线程修改失败.

但如果两次修改可交换(即两者的先后顺序不重要, 例如两次计数器累加操作), 则可以改为使用`commute`

软件事务内存
===============

基于锁的解决方案阻止超过一个线程执行受保护的部分代码。只有获得一组对应锁的线程才能执行用那些锁区别使用的代码。其他所有想要执行相同代码块的线程阻塞，直到第一个线程完成并释放那些锁。

STM系统采用几乎相反的方法。首先，需要改变数据的代码被放在一个事务中。在Clojure中，这意味着使用dosync宏。完成这一步后，语言运行时环境采用一种乐观的方法让线程执行该事务。可以有任意数量的线程开始执行事务。在事务中对ref的更改被隔离，只有做出更改的线程才能看到更改后的值。

第一个完成事务代码块执行的线程可以提交更改后的值。一旦线程提交，当其他线程试图提交时，该事务中止并回滚更改。

代理
==============

Clojure提供一种称为代理（agent）的特殊结构，可以对共享可变数据进行异步和独立更改。

使用`agent`创建代理, 使用deref解除引用(或者使用@宏)

In [18]:
(def cpu-time (agent 0))

#'user/cpu-time

In [19]:
@cpu-time

0

send
-------

代理在对特定状态的更改必须以异步方式进行时很有用。这些更改通过发送一个动作（常规的Clojure函数）给代理进行，这个动作将在以后于单独的线程上运行。

In [20]:
(send cpu-time + 700)

#agent[{:status :ready, :val 700} 0x7ad38df0]

send操作将请求提交到一个固定大小的线程池中. 如果线程池未满, 则函数立即返回.

在之后的一段时间, Clojure会调度执行对应的函数, 在执行完毕之前, 解引用依然返回旧的值.

如果提交时线程池已满, 则会阻塞send函数. 如果希望不被阻塞, 可使用send-off操作. send-off将函数提交到一个无界的线程池中, 因此永远不会阻塞.

向代理提交操作后, 可使用await或者await-for等待代理执行完毕. 

如果代理执行错误, 可以使用`agent-error`获取错误的原因. 一旦代理出现执行错误, 则后续所有的操作都是错误状态, 且代理的值也不会变换. 使用`clear-agent-errors`可以清除代理的错误状态.

事务的副作用
-----------------

Clojrue的STM系统自动重试失败的事务。在第一个事务提交之后，并发启动的其他所有事务将依次在它们试图提交时中止。然后，中止的事务重新启动。这意味着，dosync块内的代码可能需要执行多次才能成功，因此，这些代码不应该包含副作用。


原子
==============

原子是Clojure中另一种可变状态管理机制。与引用不同，原子不支持事务性更新，也不支持乐观并发控制。原子提供了一种简单的方式来管理可变状态，它使用CAS（Compare-and-Swap）操作来确保更新的原子性。原子适用于那些不需要事务性保证，但需要保证状态更新原子性的场景。

In [23]:
(def total-rows (atom 42))

#'user/total-rows

In [24]:
@total-rows

42

In [35]:
(reset! total-rows 43)

43

In [37]:
(swap! total-rows + 100)

142

监视突变
===============

Clojure支持添加监视器, 使得一个键发生变更时, 调用指定的函数

In [44]:
(def adi (atom 0))
(defn on-change [the-key the-ref old new]
    (println "On-Change" the-key the-ref old new))
(add-watch adi :adi-watcher on-change)

#atom[0 0x67ee9052]

In [45]:
@adi

0

In [46]:
(swap! adi inc)

On-Change :adi-watcher #atom[1 0x67ee9052] 0 1


1

In [47]:
(remove-watch adi :adi-watcher)

#atom[1 0x67ee9052]

Future
===========

future是代表在不同线程上执行的函数结果的一个对象. 

In [48]:
(defn slow-c [M N]
    (Thread/sleep 2000)
    (* M N))

(defn long-run []
    (let [x (slow-c 11 13)
          y (slow-c 13 17)
          z (slow-c 17 19)]
        (* x y z)))

#'user/long-run

In [49]:
(time (long-run))

"Elapsed time: 6000.576719 msecs"


10207769

In [53]:
(defn fast-run []
    (let [x (future (slow-c 11 13))
          y (future (slow-c 13 17))
          z (future (slow-c 17 19))]
        (* @x @y @z)))

#'user/fast-run

In [54]:
(time (fast-run))

"Elapsed time: 2006.047192 msecs"


10207769

使用future可以创建一个独立的线程运行给定的函数并返回一个future对象. 该操作会立刻返回. 

当后续对future进行解引用时, 会阻塞线程, 直到对应的操作执行完毕.

可以使用如下的一些方法对future进行控制

函数            | 效果
----------------|-----------
future?         | 判断一个对象是否是future对象
future-done?    | 判断是否计算结束
future-cancel?  | 如果future尚未开始则撤销操作, 否则不进行任何操作

promise
------------

promise是代表将在未来某个时点交付的一个值的对象. 可以创建一个promise对象后, 在一个线程中提交值, 在另一个线程中读取, 从而实现线程间通信.

使用`deliver`函数投递值. 使用解引用读取值. 如果promise还未被投递值, 则当前线程阻塞.

> 不要在REPL上解引用promise, 会导致阻塞